In [1]:
import pandas as pd


In [3]:
file_path = '../data/restaurant_customer_satisfaction.csv'
df = pd.read_csv(file_path)


In [6]:
df.head()

,CustomerID,Age,Gender,Income,VisitFrequency,AverageSpend,PreferredCuisine,TimeOfVisit,GroupSize,DiningOccasion,MealType,OnlineReservation,DeliveryOrder,LoyaltyProgramMember,WaitTime,ServiceRating,FoodRating,AmbianceRating,HighSatisfaction
0,654,35,Male,83380,Weekly,27.829142,Chinese,Breakfast,3,Business,Takeaway,0,1,1,43.523929,2,5,4,0
1,655,19,Male,43623,Rarely,115.408622,American,Dinner,1,Casual,Dine-in,0,0,0,57.524294,5,5,3,0
2,656,41,Female,83737,Weekly,106.693771,American,Dinner,6,Celebration,Dine-in,0,1,0,48.682623,3,4,5,0
3,657,43,Male,96768,Rarely,43.508508,Indian,Lunch,1,Celebration,Dine-in,0,0,0,7.552993,4,5,1,0
4,658,55,Female,67937,Monthly,148.084627,Chinese,Breakfast,1,Business,Takeaway,0,0,1,37.789041,2,3,5,0


In [9]:
df['Gender']=df.Gender.astype('category')
df['VisitFrequency']=df.VisitFrequency.astype('category')
df['PreferredCuisine']=df.PreferredCuisine.astype('category')
df['TimeOfVisit']=df.TimeOfVisit.astype('category')
df['DiningOccasion']=df.DiningOccasion.astype('category')
df['MealType']=df.MealType.astype('category')
df['DiningOccasion']=df.DiningOccasion.astype('category')

for col in df.columns:
    if df[col].dtype != 'category':
        df[col] = df[col].astype(float)


In [10]:
cat_columns = []
num_columns = []
bin_columns = []

def binary(col):
    return col.isin([0.0, 1.0]).all()

for col in df.columns:
    if df[col].dtype == 'category':
        cat_columns.append(col)
    elif binary(df[col]):
        bin_columns.append(col) 
    else:
        num_columns.append(col)

In [11]:
print(f'Categorical Variables: {cat_columns}')
print("------------------------------------------------------------------------")
print(f'Numerical Variables: {num_columns}')
print("------------------------------------------------------------------------")
print(f'Binary Variables: {bin_columns}')

Categorical Variables: ['Gender', 'VisitFrequency', 'PreferredCuisine', 'TimeOfVisit', 'DiningOccasion', 'MealType']
------------------------------------------------------------------------
Numerical Variables: ['CustomerID', 'Age', 'Income', 'AverageSpend', 'GroupSize', 'WaitTime', 'ServiceRating', 'FoodRating', 'AmbianceRating']
------------------------------------------------------------------------
Binary Variables: ['OnlineReservation', 'DeliveryOrder', 'LoyaltyProgramMember', 'HighSatisfaction']


In [13]:
def read_data(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)
    df['Gender']=df.Gender.astype('category')
    df['VisitFrequency']=df.VisitFrequency.astype('category')
    df['PreferredCuisine']=df.PreferredCuisine.astype('category')
    df['TimeOfVisit']=df.TimeOfVisit.astype('category')
    df['DiningOccasion']=df.DiningOccasion.astype('category')
    df['MealType']=df.MealType.astype('category')
    df['DiningOccasion']=df.DiningOccasion.astype('category')
    df['Income_per_AverageSpend'] = df['Income'] / df['AverageSpend']
    df['AverageSpend_per_GroupSize'] = df['AverageSpend'] / df['GroupSize']
    df['Income_per_GroupSize'] = df['Income'] / df['GroupSize']

    for col in df.columns:
        if df[col].dtype != 'category':
            df[col] = df[col].astype(float)
    return df
    

In [ ]:
def data_encode(df):
    